## Testing ##

In [1]:
import lxml.etree as etree
import re
from datetime import datetime

In [2]:
tree = etree.parse('xml_files/House_of_Representatives_2011_05_10_10_Official.xml')
etree.strip_tags(tree, 'inline')
root = tree.getroot()

In [3]:
# finding scheme type
for h in root.xpath("//hansard"):
    print(h.get("version"))

2.2


In [4]:
for child in root: 
    print(child.tag)

session.header
chamber.xscript
answers.to.questions


In [5]:
# defining divs
divs =  tree.findall("//chamber.xscript//debate")

In [6]:
# access session info and store in a list 
for element in root.iter('session.header'):
    for child in element:
        print("%s - %s" % (child.tag, child.text))
        
for element in root.iter('session.header'):
    for child in element.iter('date'):
        print("%s - %s" % (child.tag, child.text))

date - 2011-05-10
parliament.no - 43
session.no - 1
period.no - 3
chamber - House of Reps
page.no - 0
proof - 0
date - 2011-05-10


In [8]:
for i in root.xpath('//talk.start'):
    j = i.getparent().tag
    print(j)

speech
speech
continue
speech
speech
speech
speech
speech
speech
speech
speech
speech
speech
continue
continue
speech
speech
question
answer
question
answer
question
answer
speech
question
answer
interjection
continue
continue
continue
continue
continue
interjection
continue
continue
question
answer
continue
interjection
continue
continue
question
answer
interjection
continue
continue
continue
question
answer
question
answer
question
continue
answer
speech
continue
speech
speech
continue
interjection
continue
continue
continue
continue
continue
continue
interjection
continue
speech
speech
speech
speech
speech
speech
speech
speech
continue
continue
continue
continue
speech
speech
speech
interjection
continue
interjection
continue
speech
continue
speech
speech
speech
speech
speech
continue
speech
continue
speech
continue
continue
continue
speech
speech
speech
speech
speech
interjection
continue
speech
speech
speech
speech
speech
speech
speech
speech
speech
speech
continue
question
answer

### Names ###

In [9]:
# access speaker names, only metadata and not display names 
# if possible, get name.id, electorate, party and role as well
# this works 
for element in root.iter('talker'):
    for child in element.iter('name'):
        if child.get('role') == 'metadata':
            print("%s - %s" %(child.tag, child.text)) 

name - Gillard, Julia, MP
name - Abbott, Tony, MP
name - Jenkins, Harry, MP
name - Albanese, Anthony, MP
name - Jenkins, Harry, MP
name - Albanese, Anthony, MP
name - Gillard, Julia, MP
name - Abbott, Tony, MP
name - Jenkins, Harry, MP
name - Albanese, Anthony, MP
name - Gillard, Julia, MP
name - Abbott, Tony, MP
name - Gillard, Julia, MP
name - Gillard, Julia, MP
name - Gillard, Julia, MP
name - Abbott, Tony, MP
name - Gillard, Julia, MP
name - Abbott, Tony, MP
name - Gillard, Julia, MP
name - Abbott, Tony, MP
name - Gillard, Julia, MP
name - Brodtmann, Gai, MP
name - Gillard, Julia, MP
name - Jenkins, Harry, MP
name - Bishop, Julie, MP
name - Gillard, Julia, MP
name - Pyne, Christopher, MP
name - Jenkins, Harry, MP
name - Gillard, Julia, MP
name - Jenkins, Harry, MP
name - Gillard, Julia, MP
name - Jenkins, Harry, MP
name - Mirabella, Sophie, MP
name - Jenkins, Harry, MP
name - Gillard, Julia, MP
name - Byrne, Anthony, MP
name - Shorten, Bill, MP
name - Jenkins, Harry, MP
name - Bish

In [15]:
namemd = []
for div in divs:
    for uts in div.iter('talk.start'):
        for name in uts.xpath('talker//name'):
            if name.get('role') == 'metadata':
                namemd = name.text.split(', ')
                if len(namemd) == 1:
                    names = namemd[0]
                else:
                    names = namemd[1] + ' ' + namemd[0]
                #cleaning text
                names = re.sub(r'\([^)]*\)', '', names)
                names = names.replace('  ', ' ')
                names = names.replace('Dr ', '')
                names = names.replace('Mr ', '')
                #print(names)

### Speaker Metadata ###

In [17]:
#Dictionary
for div in divs:
    for uts in div.iter('talk.start'):
        info_dict = {}
        info_dict['nameid'] = uts.xpath('talker/name.id/text()')
        info_dict['party'] = uts.xpath('talker/party/text()')
        info_dict['electorate'] = uts.xpath('talker/electorate/text()')
        info_dict['role'] = uts.xpath('talker/role/text()')

In [23]:
#Name ID
for div in divs:
    for uts in div.iter('talk.start'):
        for nameidxp in uts.xpath('talker/name.id/text()'): info_dict['nameid'] = nameidxp
        #print(type(info_dict['nameid']))
        #print(info_dict['nameid'])

In [18]:
#Parties 
for div in divs:
    for uts in div.iter('talk.start'):
        info_dict['party'] = uts.xpath('talker/party/text()')
        #for elem in info_dict['party']:
            #print(elem)

In [9]:
#Special Roles
roles = root.xpath('//role')
for role in roles:
    print(role.text)

Prime Minister
Minister for Health and Aged Care
Prime Minister
Prime Minister
Treasurer
Prime Minister
Minister for Employment, Workplace Relations and Small Business
Prime Minister
Treasurer
Prime Minister
Deputy Prime Minister
Prime Minister
Minister for Finance and Administration
Prime Minister
Minister for Trade
Prime Minister
Minister for Health and Aged Care
Prime Minister
Minister for Education, Training and Youth Affairs and Minister Assisting the Prime Minister for the Public Service
Prime Minister
Leader of the Opposition
Prime Minister
Treasurer
Minister for Health and Aged Care
Minister for Employment, Workplace Relations and Small Business
Attorney-General
Parliamentary Secretary to the Minister for Finance and Administration
Parliamentary Secretary to the Minister for Finance and Administration
Parliamentary Secretary to the Minister for Employment, Workplace Relations and Small Business
Minister for Agriculture, Fisheries and Forestry


### Texts ### 

In [7]:
accepted_tags = ['speech', 'question', 'answer']

In [29]:
for div in divs: # each <debate> within list of <debate>s
    for uts in div.iter('talk.start'):
        if uts.getparent().tag in accepted_tags:
            for j in uts.xpath('following-sibling::talk.text'):
                span_list = []
                for k in j.iter('span'):
                    print(k.tag, k.get("class"))
                    span_list.append(k.get("class"))

span HPS-Normal
span HPS-MemberSpeech
span HPS-Electorate
span HPS-MinisterialTitles
span HPS-Time
span HPS-Small
span HPS-Normal
span HPS-Normal
span HPS-Normal
span HPS-Normal
span HPS-Normal
span HPS-Normal
span None
span None
span HPS-Normal
span HPS-Normal
span HPS-MemberSpeech
span HPS-Electorate
span HPS-MinisterialTitles
span HPS-Time
span HPS-Normal
span HPS-MemberInterjecting
span HPS-Normal
span HPS-MemberContinuation
span HPS-Normal
span None
span None
span HPS-Normal
span HPS-Normal
span HPS-MemberSpeech
span HPS-Electorate
span HPS-MinisterialTitles
span HPS-Time
span HPS-Small
span HPS-Normal
span HPS-Normal
span HPS-Normal
span HPS-MemberSpeech
span HPS-Time
span HPS-Normal
span None
span None
span HPS-Normal
span HPS-Normal
span HPS-MemberSpeech
span HPS-Electorate
span HPS-MinisterialTitles
span HPS-Time
span HPS-Normal
span None
span None
span HPS-Normal
span HPS-Normal
span HPS-MemberSpeech
span HPS-Electorate
span HPS-MinisterialTitles
span HPS-Time
span HPS-Normal

span HPS-Normal
span HPS-Normal
span HPS-Normal
span HPS-Normal
span HPS-Normal
span HPS-Normal
span HPS-MemberSpeech
span HPS-Electorate
span HPS-MinisterialTitles
span HPS-Time
span HPS-Small
span HPS-Normal
span HPS-Normal
span HPS-Normal
span HPS-Normal
span HPS-Normal
span HPS-MemberSpeech
span HPS-Electorate
span HPS-MinisterialTitles
span HPS-Time
span HPS-Normal
span HPS-Normal
span HPS-MemberContinuation
span None
span None
span HPS-Small
span HPS-Small
span HPS-Small
span HPS-Small
span HPS-Small
span HPS-Normal
span HPS-Normal
span HPS-Normal
span None
span None


In [30]:
set(span_list)

{'HPS-Electorate',
 'HPS-MemberContinuation',
 'HPS-MemberSpeech',
 'HPS-MinisterialTitles',
 'HPS-Normal',
 'HPS-Small',
 'HPS-Time',
 None}

In [24]:
# prints all text
for div in divs: # each <debate> within list of <debate>s
    for uts in div.iter('talk.start'):
        if uts.getparent().tag in accepted_tags:
            for j in uts.xpath('following-sibling::talk.text'):
                text = []
                for k in j.iter('span'):
                    if k.get('class') == "HPS-Normal" or k.get('class') == "HPS-Small":
                        if k.text:
                            if k.text:
                                text.append(k.text.strip())
                print(text)

['', 'That the House express its deep regret at the death of Mr Lionel Rose MBE, place on record its appreciation for his outstanding service to world sport and to the Australian community and tender its profound sympathy to his family and friends in their loss.', 'This week our nation marks the passing of a remarkable Australian, Lionel Rose—a man who rose from very modest beginnings to the highest level of international sport, taking the hopes of a nation with him. Lionel Rose started boxing as a 14-year-old in a makeshift boxing ring in country Victoria. It is said, according to one account, that he used to tie his hands up in rags because he could not afford the boxing gloves.', "However, within two years, he had gone professional and by 19 was bantamweight champion of the world. In that fight in 1968 he was the underdog—the underdog who prevailed brilliantly and unexpectedly in 15 rounds while thousands of Australians listened to Ron Casey's thrilling blow-by-blow live commentary 

In [37]:
# prints all tails
for div in divs: # each <debate> within list of <debate>s
    for uts in div.iter('talk.start'):
        if uts.getparent().tag in accepted_tags:
            for j in uts.xpath('following-sibling::talk.text'):
                text2 = []
                for k in j.iter('span'):
                    if k.tail:
                        text2_ed = re.sub('\)\:', '', k.tail.strip()).strip()
                        text2.append(text2_ed)
                print(text2)

['', '', '—', ') (', 'I move:', '', '', '', '', '', '', '', 'and', '—songs which reinforced his fame with many and introduced him to new fans across Australia.', '']
['', '', '—', ') (', 'I rise to join the Prime Minister in remembering a great Australian. Lionel Rose was never really a political activist, but few have equalled his efforts to bring about true reconciliation. When he returned from his legendary win against Fighting Harada in Tokyo in 1968, a quarter of a million Australians lined the streets to cheer. It was the first time so many Australians had cheered an Aboriginal person. How could Aboriginal people not be first-class citizens in their own country when one of them had done us all so proud? This was, I suspect, a seminal moment in the history of race relations in this country. As the Prime Minister said, we do remember him—especially his never-say-die spirit, which will always be an important part of the Australian character.', '', 'Hear, hear!', '', '', '', '', '', 

In [36]:
re.sub('\)\:', '', '):  I move:'.strip()).strip()

'I move:'

In [40]:
# prints all tails with class filter
for div in divs: # each <debate> within list of <debate>s
    for uts in div.iter('talk.start'):
        if uts.getparent().tag in accepted_tags:
            for j in uts.xpath('following-sibling::talk.text'):
                text2 = []
                for k in j.iter('span'):
                    if k.get('class') == "HPS-Normal" or k.get('class') == "HPS-Small" or k.get('class') == None or k.get('class') == "HPS-Time":
                        if k.tail:
                            text2.append(k.tail.strip())
                print(text2)

['', '):  I move:', '', '', '', '', '', '', '', 'and', '—songs which reinforced his fame with many and introduced him to new fans across Australia.', '']
['', '):  I rise to join the Prime Minister in remembering a great Australian. Lionel Rose was never really a political activist, but few have equalled his efforts to bring about true reconciliation. When he returned from his legendary win against Fighting Harada in Tokyo in 1968, a quarter of a million Australians lined the streets to cheer. It was the first time so many Australians had cheered an Aboriginal person. How could Aboriginal people not be first-class citizens in their own country when one of them had done us all so proud? This was, I suspect, a seminal moment in the history of race relations in this country. As the Prime Minister said, we do remember him—especially his never-say-die spirit, which will always be an important part of the Australian character.', '', '', '', '', '', '']
['', '):  by leave—I move:', '', '', ''

In [9]:
# combine?
for div in divs: # each <debate> within list of <debate>s
    for uts in div.iter('talk.start'):
        if uts.getparent().tag in accepted_tags:
            # speaker
            for name in uts.xpath('talker//name'):
                if name.get('role') == 'metadata':
                    namemd = name.text.split(', ')
                    if len(namemd) == 1:
                        names = namemd[0]
                    else:
                        names = namemd[1] + ' ' + namemd[0]
                    print('SPEAKER:',names)
            # text
            for j in uts.xpath('following-sibling::talk.text'):
                text3 = []
                for k in j.iter('span'):
                    if k.get('class') == "HPS-Normal" or k.get('class') == "HPS-Small" or k.get('class') == None:
                        if k.text is not None:
                            text3.append(k.text.strip())
                    if k.get('class') == "HPS-Normal" or k.get('class') == "HPS-Small" or k.get('class') == None or k.get('class') == "HPS-Time":
                        if k.tail:
                            text3.append(k.tail.strip())
                print(text3)

SPEAKER: Julia Gillard
['', '', '):  I move:', 'That the House express its deep regret at the death of Mr Lionel Rose MBE, place on record its appreciation for his outstanding service to world sport and to the Australian community and tender its profound sympathy to his family and friends in their loss.', '', 'This week our nation marks the passing of a remarkable Australian, Lionel Rose—a man who rose from very modest beginnings to the highest level of international sport, taking the hopes of a nation with him. Lionel Rose started boxing as a 14-year-old in a makeshift boxing ring in country Victoria. It is said, according to one account, that he used to tie his hands up in rags because he could not afford the boxing gloves.', '', "However, within two years, he had gone professional and by 19 was bantamweight champion of the world. In that fight in 1968 he was the underdog—the underdog who prevailed brilliantly and unexpectedly in 15 rounds while thousands of Australians listened to R

In [26]:
# finding interjections 
# prints all tails
for div in divs: # each <debate> within list of <debate>s
    for uts in div.iter('talk.start'):
        if uts.getparent().tag in accepted_tags:
            for j in uts.xpath('following-sibling::talk.text'):
                for k in j.iter('span'):
                    for a in k.iter('a'):
                        if a.get("type") == "MemberInterjecting":
                            print(a.get("href"))
                            for name in a.iter('span'): print(name.text.strip())
                            print(a.tail.strip())

9V5
Mr Pyne:
On a point of order, Mr Speaker: the Prime Minister was asked a very straightforward question without any hyperbole and I would ask you to ask her to answer it directly.
00AMU
Mrs Mirabella:
One royal wedding invitation and you think you are a queen.
83P
Ms Julie Bishop:
I withdraw.
E3L
Mr Morrison:
Mr Speaker, on a point of order: it was a very specific question. Were they allowed to come down from the roof and then go back without interference?
EZ5
Mr Abbott:
Reluctantly, Mr Speaker. I would imagine that this is way outside the standing orders. It has nothing to do with the subject matter of the suspension and it has nothing to do with why it is urgent.
83V
Dr Emerson:
They are against that.
E0J
Mr Keenan:
The money is so good.
E0J
Mr Keenan:
Lucky they did not burn things down.
HVP
Mr Perrett:
Premiers.


In [20]:
# finding interjections 
# prints all tails
for div in divs: # each <debate> within list of <debate>s
    for uts in div.iter('talk.start'):
        if uts.getparent().tag in accepted_tags:
            for j in uts.xpath('following-sibling::talk.text'):
                for k in j.iter('span'):
                    for a in k.iter('a'):
                        if a.get("type") == "MemberContinuation":
                            print("Speaker ID: {}".format(a.get("href")))
                            print(a.tail.strip())
                            print("\n")

Speaker ID: HH4
As a mark of respect, I invite honourable members to rise in their places.


Speaker ID: 83L
I thank members of the opposition for their sterling support. This was, of course, a highly formal and symbolic occasion for the Royal Family, for the British system of government and for the entire Commonwealth. It was also a very human occasion: two young people, very much in love, having the courage to build a life together amid the glare of publicity and scrutiny.


Speaker ID: 83L
They are romantics on the opposition frontbench, Mr Speaker. It is obviously a huge pressure to know that your wedding will be watched by probably two billion people—that is an amazing statistic. The upside of that pressure is the joy that is given to so many. I had the opportunity to see that joy personally in the abbey and beyond with other prime ministers and governors-general. We watched the crowd outside Buckingham Palace as they saw William and Kate and the Royal Family come out onto the bal

---